# Traffic Estimation Using Sublinear Space

### (a) Estimating the Number of Unique IP Addresses

#### Approach Used
We use two techniques:
- **Exact method**: Store all IPs in a Python `set` to count unique entries (linear space)
- **Approximate method**: Use the HyperLogLog algorithm with `p=10` (sublinear space)

#### Explanation
HyperLogLog is a probabilistic algorithm that estimates the number of distinct elements without storing them. It works by:
- Hashing each IP address to a 32-bit number
- Using the first `p` bits to select a register
- Counting the number of leading zeros in the remaining bits
- Storing the maximum number of leading zeros seen per register
- Combining all register values using a harmonic mean and a correction factor `alpha` to produce the estimate

In [1]:
import mmh3
import math

def hll_init(p=10):
    m = 1 << p
    return {
        'p': p,
        'm': m,
        'alpha': 0.7213 / (1 + 1.079 / m),
        'registers': [0] * m
    }

def hll_update(hll, value):
    x = mmh3.hash(str(value), signed=False)
    j = x >> (32 - hll['p'])
    w = x & ((1 << (32 - hll['p'])) - 1)
    rank = (32 - hll['p']) - w.bit_length() + 1 if w > 0 else (32 - hll['p']) + 1
    hll['registers'][j] = max(hll['registers'][j], rank)

def hll_count(hll):
    Z = 1 / sum([2.0 ** -r for r in hll['registers']])
    E = hll['alpha'] * hll['m'] * hll['m'] * Z
    V = hll['registers'].count(0)
    if E <= 2.5 * hll['m'] and V > 0:
        E = hll['m'] * math.log(hll['m'] / V)
    return int(E)

#### Steps
1. Load source and destination IPs from the dataset
2. Combine all IPs into a single list
3. Use a Python `set()` to count exact unique IPs (requires storing all IPs)
4. Initialize the HyperLogLog structure with `p=10` to define 1024 registers
5. For each IP, hash and update the appropriate register based on leading zeros
6. Compute the final estimate using the HLL formula with correction factor
7. Calculate the error percentage between exact and estimated counts
8. Compare time and memory used by both methods

In [2]:
import pandas as pd
import time
import sys
import glob

csv_files = glob.glob("data/*.csv")
df = pd.concat([pd.read_csv(f, usecols=['source', 'destination']) for f in csv_files], ignore_index=True)
all_ips = pd.concat([df['source'], df['destination']], ignore_index=True)

start = time.time()
unique_ips_exact = len(set(all_ips))
exact_time = time.time() - start
exact_memory = sys.getsizeof(set(all_ips)) + sum(sys.getsizeof(ip) for ip in set(all_ips))

hll = hll_init(p=10)
start = time.time()
for ip in all_ips:
    hll_update(hll, ip)
unique_ips_estimated = hll_count(hll)
hll_time = time.time() - start

error_percent = abs(unique_ips_exact - unique_ips_estimated) / unique_ips_exact * 100
hll_memory = sys.getsizeof(hll['registers']) + sys.getsizeof(hll)

print(f"Exact unique IPs: {unique_ips_exact}")
print(f"Estimated unique IPs: {unique_ips_estimated}")
print(f"Error rate: {error_percent:.2f}%")
print(f"Exact time (s): {exact_time:.4f}")
print(f"HLL time (s): {hll_time:.4f}")
print(f"Exact memory (KB): {exact_memory / 1024:.2f}")
print(f"HLL memory (KB): {hll_memory / 1024:.2f}")

Exact unique IPs: 34801
Estimated unique IPs: 32034
Error rate: 7.95%
Exact time (s): 0.1349
HLL time (s): 1.0584
Exact memory (KB): 3884.00
HLL memory (KB): 8.23


### (b) Identifying Frequently Contacted Destination IPs

#### Approach Used
We use two techniques:
- **Exact method**: Python `Counter` to count the frequency of each destination IP (linear space)
- **Approximate method**: Count-Min Sketch (sublinear space)

#### Explanation
Count-Min Sketch is a probabilistic data structure that estimates frequency of elements using:
- A 2D array of counters
- Multiple hash functions
- Each input updates one counter per row (based on each hash)
- To estimate the frequency, it returns the minimum count from all hash rows

This method avoids storing per-IP counters and uses fixed memory, at the cost of a small overestimation.

In [3]:
import numpy as np

def cms_init(width=10000, depth=5):
    return {
        'width': width,
        'depth': depth,
        'table': np.zeros((depth, width), dtype=int)
    }

def cms_hash(x, i, width):
    return mmh3.hash(str(x), seed=i) % width

def cms_update(cms, x):
    for i in range(cms['depth']):
        idx = cms_hash(x, i, cms['width'])
        cms['table'][i, idx] += 1

def cms_estimate(cms, x):
    return min(cms['table'][i, cms_hash(x, i, cms['width'])] for i in range(cms['depth']))

### Steps

1. Load the destination IPs from the dataset.
2. Use a Python `Counter` to count exact frequencies for each destination IP.
3. Initialize the Count-Min Sketch with fixed `width` and `depth`.
4. For each destination IP, update the CMS table using multiple hash functions.
5. Identify the top destination IPs from the exact method.
6. Estimate the same IPs’ frequencies using CMS.
7. Compute the error for each IP and calculate the average error.
8. Measure and compare the time and memory used by both methods.

In [4]:
from collections import Counter

df = pd.concat([pd.read_csv(f, usecols=['destination']) for f in csv_files], ignore_index=True)
dest_ips = df['destination']

start = time.time()
exact_counts = Counter(dest_ips)
top_exact = exact_counts.most_common(10)
exact_time = time.time() - start
exact_memory = sys.getsizeof(exact_counts) + sum(sys.getsizeof(ip) + sys.getsizeof(count) for ip, count in exact_counts.items())

cms = cms_init(width=10000, depth=5)
start = time.time()
for ip in dest_ips:
    cms_update(cms, ip)
cms_time = time.time() - start
cms_memory = cms['table'].nbytes + sys.getsizeof(cms)

cms_estimates = [(ip, cms_estimate(cms, ip)) for ip, _ in top_exact]
errors = [abs(est - actual) / actual * 100 for (ip, actual), (_, est) in zip(top_exact, cms_estimates)]
avg_error = sum(errors) / len(errors)

print(f"Exact top 10 destination IPs:")
for ip, count in top_exact:
    print(f"{ip}: {count}")

print("\nCMS estimates for top 10:")
for ip, est in cms_estimates:
    print(f"{ip}: {est}")

print(f"\nAverage error: {avg_error:.2f}%")
print(f"Exact time (s): {exact_time:.4f}")
print(f"CMS time (s): {cms_time:.4f}")
print(f"Exact memory (KB): {exact_memory / 1024:.2f}")
print(f"CMS memory (KB): {cms_memory / 1024:.2f}")

Exact top 10 destination IPs:
198.164.30.2: 232409
192.168.5.122: 199437
203.73.24.75: 193200
125.6.164.51: 106826
67.220.214.50: 49298
202.210.143.140: 36189
82.98.86.183: 25214
95.211.98.12: 25095
209.112.44.10: 21824
62.140.213.243: 20509

CMS estimates for top 10:
198.164.30.2: 232409
192.168.5.122: 199439
203.73.24.75: 193202
125.6.164.51: 106839
67.220.214.50: 49300
202.210.143.140: 36198
82.98.86.183: 25215
95.211.98.12: 25096
209.112.44.10: 21826
62.140.213.243: 20515

Average error: 0.01%
Exact time (s): 0.1169
CMS time (s): 2.5796
Exact memory (KB): 3706.48
CMS memory (KB): 390.80


### (c) Efficient Membership Testing for Blocklists

#### Approach Used
We use two techniques:
- **Exact method**: Use a Python `set()` to store and test previously seen IPs (linear space)
- **Approximate method**: Use a Bloom filter for memory-efficient membership testing (sublinear space)

#### Explanation
A Bloom filter is a probabilistic data structure used to test whether an element is in a set. It uses:
- A fixed-size bit array
- Multiple hash functions

To add an IP, the Bloom filter sets multiple bits in the array based on the hash functions.  
To check membership, it verifies if all those bits are set.  
If they are, the element is *probably* in the set. If any are unset, the element is *definitely not* in the set.

Bloom filters can have **false positives** (reporting an unseen IP as seen) but never false negatives.  
This makes them useful when space is limited and occasional errors are acceptable.

In [5]:
import mmh3
import numpy as np

def bloom_init(size=10000, num_hashes=3):
    return {
        'size': size,
        'num_hashes': num_hashes,
        'bit_array': np.zeros(size, dtype=bool)
    }

def bloom_hashes(value, bloom):
    return [mmh3.hash(str(value), seed=i) % bloom['size'] for i in range(bloom['num_hashes'])]

def bloom_add(bloom, value):
    for h in bloom_hashes(value, bloom):
        bloom['bit_array'][h] = True

def bloom_check(bloom, value):
    return all(bloom['bit_array'][h] for h in bloom_hashes(value, bloom))

### Steps

1. Load a sample of IP addresses from the dataset.
2. Use a Python `set()` to store and test IPs exactly.
3. Initialize a Bloom filter with fixed size and number of hash functions.
4. Add a portion of the IPs to both the set and Bloom filter.
5. Use another portion as test IPs to check for membership.
6. For each test IP, check membership using both methods.
7. Count how often Bloom filter says “yes” but set says “no” → false positives.
8. Calculate false positive rate and compare time and memory usage.

In [6]:
import random

df = pd.concat([pd.read_csv(f, usecols=['source', 'destination']) for f in csv_files], ignore_index=True)
all_ips = pd.concat([df['source'], df['destination']], ignore_index=True).unique()
random.shuffle(all_ips)

train_ips = all_ips[:10000]
test_ips = all_ips[10000:20000]

start = time.time()
exact_set = set(train_ips)
exact_time = time.time() - start
exact_memory = sys.getsizeof(exact_set) + sum(sys.getsizeof(ip) for ip in exact_set)

bloom = bloom_init(size=100000, num_hashes=4)
start = time.time()
for ip in train_ips:
    bloom_add(bloom, ip)
bloom_time = time.time() - start
bloom_memory = bloom['bit_array'].nbytes + sys.getsizeof(bloom)

false_positives = 0
total_tests = len(test_ips)

for ip in test_ips:
    if bloom_check(bloom, ip) and ip not in exact_set:
        false_positives += 1

false_positive_rate = (false_positives / total_tests) * 100

print(f"False positive rate: {false_positive_rate:.2f}%")
print(f"Exact time (s): {exact_time:.4f}")
print(f"Bloom time (s): {bloom_time:.4f}")
print(f"Exact memory (KB): {exact_memory / 1024:.2f}")
print(f"Bloom memory (KB): {bloom_memory / 1024:.2f}")

False positive rate: 1.38%
Exact time (s): 0.0004
Bloom time (s): 0.0063
Exact memory (KB): 1039.74
Bloom memory (KB): 97.84
